In [ ]:
# 파이썬 프로그램 자동채점 (파일상태 검사) AutoGrader V1.1
# Copyrighted by Young Pyo Jun, ypjun@yonsei.ac.kr
# V1.0 (2021.12.19)

#-*-encoding:utf-8-*-
import pandas as pd
import os
import re
import tkinter as tk
from tkinter import filedialog
import subprocess as sp
import sys

from colorama import init, Fore, Back, Style

# 채점 프로그램이 있는 폴더를 선택
def SelectFile():
    fname = ""
    window = tk.Tk()
    window.tkraise()
    window.file = filedialog.askdirectory(initialdir=".", title="Select")
    if window.file != None:
        fname = window.file
    window.destroy()
    return fname

def GetPNum(pName):
    if pName.split('.')[-1] == 'zip': return -1
    if pName.split('.')[-1] != 'py': return 0
    nPS = pName.find('(')
    nPE = pName.find(')')
    if nPS == -1 or nPE == -1:
        n = -1
    else:
        st = pName[nPS+1:nPE]
        if st.isdigit():
            n = int(st)
        else:
            n = -1
    return n  

#################################################################################################
HomeDir = SelectFile()    # 분반 폴더 선택
if HomeDir:
    class_name = HomeDir.split('/')[-1]
    os.chdir(HomeDir)
    stdDir = HomeDir + '/응시자'

    scrFile = "ScoringTable.xlsx"
#    scrFile = "Scoring.xlsx"
    scrFrame = pd.read_excel(scrFile, dtype=str)
    stFolders = os.listdir(stdDir)
else:
    stFolders = []
    print("Exit...")

maxpNum = len(scrFrame)

np_list = ("os", "subprocess", "sys", "popen", "call", "run",
           "chdir", "mkdir", "exec[]*", "remove", "rmdir", "unlink")
np_cut = ()
#np_cut = ("import os")
def check_np(pfile):
    fp = open(pfile, "rt", encoding='utf-8')
    ct = fp.read()
    fp.close()
    nps = []
    for wd in np_list:
        if wd in np_cut: continue
        #if wd in ct:
        if re.search(f"^{wd}[^_A-Za-z]|[^_A-Za-z]{wd}[^_A-Za-z]", ct):
            nps.append(wd)
    return nps

print(f"Starting to check ({class_name}) ======================================\n")
for stf in stFolders:               # Checking Student
    stName, stId = stf.split('-')
    stId = int(stId.split('_')[0])
    stDir = stdDir+'/'+stf
    pyFiles = os.listdir(stDir)
#    ans = input("Checking for {}({})? [a for all skip, q for Finishing, n for next, 이름 for searching]".format(stName, stId))
    pNums = []
    for pyF in pyFiles:             # 각 문제 채점 (문제번호=??)
        pNum = GetPNum(pyF)
        npl = check_np(f"{stdDir}/{stf}/{pyF}")
        if npl:
            print("Dangers in {} - {}".format(stName, stf))
            print("File: {} include {}\n".format(pyF, npl))
        if pNum == 0: continue
        if pNum < 1 or pNum > maxpNum or pNum in pNums:
            print("Numbering error in {} - {}".format(stName, stf))
            print("Files: {}\n".format(pyFiles))
            break
        pNums.append(pNum)
print(f"check finished ({class_name}) =========================================")